# DO NOT EDIT: Libraries & Global Parameters

In [2]:
import random, json, math, statistics, textwrap
import pandas as pd
from collections import defaultdict
from dataclasses import dataclass, field
from openai import OpenAI
from re import TEMPLATE

!pip install -q autogen~=0.2 > /dev/null 2>&1

try:
    from autogen import AssistantAgent, GroupChatManager, GroupChat
except ModuleNotFoundError as e:
    raise ImportError("The 'autogen' module is not installed. Please install it using 'pip install pyautogen'.") from e

MODEL = "gpt-5-mini"
API_KEY = ""
BASE_URL = "https://api.openai.com/v1"

llm_config = {
    "model": MODEL,
    "api_key": API_KEY,
    "base_url": BASE_URL,
}

client = OpenAI(
    api_key=API_KEY,
    base_url=llm_config["base_url"]
)

random.seed(42)

# EDIT: Overview of All Attributes

Here, you can view and edit all attributes (incl. their anchors) that we can use for later simulations and persona construction.

In [3]:
# 1) Personal attributes
ATTRS = {
    # --- Surface-level attributes (readily observable or easily inferred) ---
    "age":                {"min":0,"max":100,"low":"very young (≈18)","high":"older (≈70)"},
    "gender_expression":  {"min":0,"max":100,"low":"traditionally feminine","high":"traditionally masculine"},
    "education":          {"min":0,"max":100,"low":"no formal education","high":"very high (PhD/prof.)"},
    "occupation_status":  {"min":0,"max":100,"low":"unemployed / entry-level","high":"senior professional / executive"},
    "income":             {"min":0,"max":100,"low":"very low income","high":"very high income/wealth"},
    "urbanicity":         {"min":0,"max":100,"low":"very rural","high":"very urban (dense metro)"},
    "ethnocultural_identity": {"min":0,"max":100,"low":"monocultural / traditional","high":"multicultural / global"},
    "communication_style": {"min":0,"max":100,"low":"reserved / indirect","high":"expressive / direct"},

    # --- Deep-level attributes (values, beliefs, personality, cognition) ---
    "political_ideology": {"min":0,"max":100,"low":"very liberal / progressive","high":"very conservative / traditional"},
    "religiosity":        {"min":0,"max":100,"low":"non-religious","high":"highly devout"},
    "cultural_openness":  {"min":0,"max":100,"low":"insular / local","high":"cosmopolitan / multilingual"},
    "risk_taking":        {"min":0,"max":100,"low":"risk-averse","high":"risk-seeking"},
    "extraversion":       {"min":0,"max":100,"low":"very introverted","high":"very extraverted"},
    "agreeableness":      {"min":0,"max":100,"low":"abrasive / competitive","high":"cooperative / warm"},
    "conscientiousness":  {"min":0,"max":100,"low":"disorganized / spontaneous","high":"disciplined / structured"},
    "openness":           {"min":0,"max":100,"low":"closed to novelty","high":"highly open to ideas / experiences"},
    "creativity":         {"min":0,"max":100,"low":"prefers convention","high":"original / imaginative"},
    "moral_foundation":   {"min":0,"max":100,"low":"care/fairness emphasis","high":"loyalty/authority emphasis"},
    "values_independence":{"min":0,"max":100,"low":"prefers interdependence","high":"prefers autonomy / self-direction"},
    "work_centrality":    {"min":0,"max":100,"low":"job is a means","high":"job is core identity / purpose"},
    "tech_adoption":      {"min":0,"max":100,"low":"avoids new tech","high":"early adopter / tech enthusiast"},
    "environmentalism":   {"min":0,"max":100,"low":"unconcerned","high":"very concerned / active"},
    "intellectual_style": {"min":0,"max":100,"low":"concrete / practical thinker","high":"abstract / theoretical thinker"},
    "emotional_expressivity":{"min":0,"max":100,"low":"restrained / stoic","high":"emotionally expressive / open"},
    "humor_style":        {"min":0,"max":100,"low":"dry / sarcastic","high":"playful / affiliative"},
}

## DO NOT EDIT: Helper to Create Agents

In [4]:
# Active personal attribute selection for this run
ACTIVE_ATTRS = []  # filled by set_active_attrs()

def set_active_attrs(attrs):
    """Declare which attributes are active for this run, in the order that values will be provided."""
    missing = [a for a in attrs if a not in ATTRS]
    if missing:
        raise ValueError(f"Unknown attribute(s): {missing}")
    if len(set(attrs)) != len(attrs):
        raise ValueError("Duplicate attributes in active set.")
    global ACTIVE_ATTRS
    ACTIVE_ATTRS = list(attrs)

# Utilities
_NA_STRINGS = {"na", "n/a", ""}

def _is_na(x):
    if x is None:
        return True
    if isinstance(x, float) and math.isnan(x):
        return True
    if isinstance(x, str) and x.strip().lower() in _NA_STRINGS:
        return True
    return False

def _validate(attr, value):
    lo, hi = ATTRS[attr]["min"], ATTRS[attr]["max"]
    if not (lo <= value <= hi):
        raise ValueError(f"Value for {attr}={value} must be in [{lo}, {hi}]")

# Agent representation
@dataclass(frozen=True)
class Agent:
    name: str
    attributes: dict = field(default_factory=dict)

# Core creator: supply exactly len(ACTIVE_ATTRS) values in order
def create_agent_from_values(values, name=None):
    """
    values: iterable with length == len(ACTIVE_ATTRS)
            Use NA markers (None, '', 'NA', 'n/a', float('nan')) to skip an attribute.
    """
    if not ACTIVE_ATTRS:
        raise RuntimeError("ACTIVE_ATTRS is empty. Call set_active_attrs([...]) first.")
    values = list(values)
    if len(values) != len(ACTIVE_ATTRS):
        raise ValueError(f"Expected {len(ACTIVE_ATTRS)} values, got {len(values)}.")

    agent_attrs = {}
    for attr, val in zip(ACTIVE_ATTRS, values):
        if _is_na(val):
            # leave attribute undefined (skip)
            continue
        if not isinstance(val, (int, float)):
            raise TypeError(f"Non-numeric value for {attr}: {val!r}")
        _validate(attr, float(val))
        agent_attrs[attr] = float(val)

    return Agent(name=name or "agent", attributes=agent_attrs)

## DO NOT EDIT: Helper for Output Overview

In [5]:
def compare_agents(reference_agent, other_agents, active_attrs=None):
    """
    Display the original attribute values (not distances) for each agent as text.
    """
    if active_attrs is None:
        active_attrs = ACTIVE_ATTRS

    if not active_attrs:
        raise ValueError("No active attributes to display. Did you call set_active_attrs([...])?")

    # Build header
    agent_names = [reference_agent.name] + list(other_agents.keys())
    header = ["Attribute"] + agent_names
    rows = []

    # Build rows for each attribute
    for a in active_attrs:
        vals = [reference_agent.attributes.get(a)]
        for _, agent in other_agents.items():
            vals.append(agent.attributes.get(a))
        rows.append([a] + vals)

    # Determine column widths for pretty printing
    col_widths = [max(len(str(x)) for x in col) + 2 for col in zip(header, *rows)]

    # Helper for safe formatting
    def fmt(val):
        if val is None or (isinstance(val, float) and math.isnan(val)):
            return "NA"
        return f"{val:.1f}" if isinstance(val, (int, float)) else str(val)

    # Print table header
    print("\n\n=== Actual Attribute Values ===\n")
    print("".join(h.ljust(w) for h, w in zip(header, col_widths)))
    print("-" * sum(col_widths))

    # Print each row
    for r in rows:
        print("".join(fmt(x).ljust(w) for x, w in zip(r, col_widths)))

# SIM 1: Similarity Perceptions

In this simulation, we supply quantitative attribute scores to create 2 personas with an LLM. Then, we let one generative agent be one of the personas and let them evaluate in terms of similarity the other persona.

Broadly speaking, we can examine based on the alignment of which attributes a generative agent forms perceptions of similarity.

However, we should be mindful that in real world, narrative descriptions of others will not be easily available, and we have to form our impressions of others based on cues we observe. Thus, we might consider this simulation a "baseline" test of similarity perceptions in an (even more) artificial setting.

## EDIT: Define Attributes

Here, you pick the attributes that you want Persona 1 (the focal agent) and Persona 2 to have.

You can add more/different attributes, as well as different values for the attributes, if desired.

In [37]:
# Pick the active attributes
set_active_attrs(["conscientiousness", "agreeableness", "openness", "extraversion", "income"])

# Set values for active attributes; use "NA" to leave an attribute undefined.
persona_1_sim1     = create_agent_from_values([30, 50, 50, 50, 30], name="persona_1")
persona_2_sim1     = create_agent_from_values([70, 50, 50, 50, 70], name="persona_2")

## DO NOT EDIT: LLM Prompt for Narratives

In [38]:
# Concat
agents_text_sim1 = "\n\nPersona 1's personality:\n" + "\n".join(
    [
        f"  - {a}: {persona_1_sim1.attributes.get(a, 'UNDEFINED')}/100  (0={ATTRS[a]['low']}, 100={ATTRS[a]['high']})"
        for a in ACTIVE_ATTRS
    ]
)
agents_text_sim1 += "\n\nPersona 2's personality:\n" + "\n".join(
    [
        f"  - {a}: {persona_2_sim1.attributes.get(a, 'UNDEFINED')}/100  (0={ATTRS[a]['low']}, 100={ATTRS[a]['high']})"
        for a in ACTIVE_ATTRS
    ]
)

task_prompt_narrative_sim1 = """Convert numeric profiles into grounded and detailed personality narratives (10 sentences each).
Use the anchors to interpret direction.
Narrative must not contain EXPLICITLY information about the anchors or scales that it were used to generate it (e.g., no mention of provided scores/provided attributes/etc.).
Instead, each narrative should IMPLICITLY reflect the numeric profiles and their anchors."""

output_format_narrative_sim1 = """Return STRICT JSON with three string fields (no markdown): {{"persona_1": "<10 sentences in SECOND PERSON (you/yours/etc.)>", "persona_2": "<10 sentences in THIRD PERSON (they/them; NO first-person>"}}"""

PROMPT_narrative_sim1 = task_prompt_narrative_sim1 + agents_text_sim1 + "\n\n" + output_format_narrative_sim1

print(PROMPT_narrative_sim1)

Convert numeric profiles into grounded and detailed personality narratives (10 sentences each).
Use the anchors to interpret direction.
Narrative must not contain EXPLICITLY information about the anchors or scales that it were used to generate it (e.g., no mention of provided scores/provided attributes/etc.).
Instead, each narrative should IMPLICITLY reflect the numeric profiles and their anchors.

Persona 1's personality:
  - conscientiousness: 30.0/100  (0=disorganized / spontaneous, 100=disciplined / structured)
  - agreeableness: 50.0/100  (0=abrasive / competitive, 100=cooperative / warm)
  - openness: 50.0/100  (0=closed to novelty, 100=highly open to ideas / experiences)
  - extraversion: 50.0/100  (0=very introverted, 100=very extraverted)
  - income: 30.0/100  (0=very low income, 100=very high income/wealth)

Persona 2's personality:
  - conscientiousness: 70.0/100  (0=disorganized / spontaneous, 100=disciplined / structured)
  - agreeableness: 50.0/100  (0=abrasive / competit

## DO NOT EDIT: LLM Request for Narratives

In [39]:
# Request
desc_sim1 = client.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": "You convert numeric profiles into faithful narratives. Output valid JSON only."},
        {"role": "user", "content": PROMPT_narrative_sim1}
    ],
).choices[0].message.content.strip()

# Output
try:
    narratives_sim1 = json.loads(desc_sim1)
except json.JSONDecodeError:
    s, e = desc_sim1.find("{"), desc_sim1.rfind("}")
    narratives_sim1 = json.loads(desc_sim1[s:e+1])

print(json.dumps(narratives_sim1, indent=2))

{
  "persona_1": "You tend to prefer a flexible approach to plans, often changing course when something more interesting appears. Deadlines can feel like suggestions, and you sometimes rely on bursts of energy to finish tasks at the last minute. You are pleasant in conversation, balancing directness with warmth so that people feel heard without you losing your footing. Social situations suit you when they are low-pressure, and you move easily between groups without needing to be the center of attention. You enjoy familiar comforts and small novelties in equal measure, liking a mix of routine and surprise. When money is tight you become resourceful, finding practical ways to make things work without unnecessary expenditure. You value authenticity and tend to speak plainly, even when that means foregoing politeness for clarity sometimes. You learn best by doing, preferring hands-on problem solving over abstract theory. Your workspace can be cluttered and personal, with useful chaos that 

## VIEW (DO NOT EDIT): LLM Prompt for Similarity Task

Here, you can view the prompts that we use to define for the LLM the task and the output format.

This prompt will include the previously generated narratives.

In [40]:
# Task and Output Prompts
task_prompt_sim1 = (
    "Below is a short narrative describing another person.\n"
    "Your task is to evaluate how similar this person feels to you overall."
)

output_format_sim1 = (
    "OUTPUT FORMAT (must follow exactly)\n"
    "- First token: a similarity score, i.e., percentage between 0% and 100%, "
    "where 0% = completely dissimilar and 100% = extremely similar\n"
    "- Then: ' — ' (em dash surrounded by spaces)\n"
    "- Then: one concise sentence explaining the judgment.\n\n"
    "Now make your judgment and explain on a single line as specified."
)

# Narratives
narratives_text_sim1_focal = f"{narratives_sim1['persona_1']}"

narratives_text_sim1_other = "\n\n" + f"Other person:\n{narratives_sim1['persona_2']}\n\n"

# Combine
PROMPT_sim1 = narratives_text_sim1_focal + "\n\n" + task_prompt_sim1 + narratives_text_sim1_other + output_format_sim1

print(PROMPT_sim1)

You tend to prefer a flexible approach to plans, often changing course when something more interesting appears. Deadlines can feel like suggestions, and you sometimes rely on bursts of energy to finish tasks at the last minute. You are pleasant in conversation, balancing directness with warmth so that people feel heard without you losing your footing. Social situations suit you when they are low-pressure, and you move easily between groups without needing to be the center of attention. You enjoy familiar comforts and small novelties in equal measure, liking a mix of routine and surprise. When money is tight you become resourceful, finding practical ways to make things work without unnecessary expenditure. You value authenticity and tend to speak plainly, even when that means foregoing politeness for clarity sometimes. You learn best by doing, preferring hands-on problem solving over abstract theory. Your workspace can be cluttered and personal, with useful chaos that reflects how your 

## DO NOT EDIT: LLM Request for Narratives

In [41]:
# Request
response_sim1 = client.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": "You are a precise evaluator who follows formatting instructions exactly."},
        {"role": "user", "content": PROMPT_sim1}
    ]
)

# Output
result_sim1 = response_sim1.choices[0].message.content.strip()

## VIEW (DO NOT EDIT): Results for Similarity Task

Here, you can view the results of our simulation. At the top of the output, you see the agent's similarity perception of the other persona. At the bottom, you can see the actual attribute values that you supplied when constructing the personas. Compare the two!

In [42]:
print(result_sim1)
compare_agents(persona_1_sim1, {"persona_2 (other person)": persona_2_sim1})

45% — They share interpersonal reliability and a readiness to take responsibility, but differ sharply in planning, organization, financial habits, and tolerance for structure and deadlines.


=== Actual Attribute Values ===

Attribute          persona_1  persona_2 (other person)  
--------------------------------------------------------
conscientiousness  30.0       70.0                      
agreeableness      50.0       50.0                      
openness           50.0       50.0                      
extraversion       50.0       50.0                      
income             30.0       70.0                      


# SIM 2: Hiring Decisions

In this simulation, similarly to SIM 1, we supply quantitative attribute scores to create 3 personas with an LLM. Then, we let one generative agent be one of the personas and let them evaluate as a hiring manager the two other personas.

Broadly speaking, we can examine whether the alignment of specific attributes between hiring manager agent and candidates influences the hiring manager agent's hiring decisions. By prompting the agent for a reason for the decision, we can also qualitatively examine whether similarity perceptions and/or homophily plays a role in these artificial hiring decisions.

Thus, while similar to SIM 1, we now introduce a clearly defined context and goal (hiring the better candidate) to the simulation and can explore whether this goal influences similarity perceptions and/or homophily.

Again, we should be mindful that in real world, narrative descriptions of others will not be easily available, and we have to form our impressions of others based on cues we observe. Thus, we might also consider this simulation a "baseline" test of similarity perceptions in an (even more) artificial setting.

## EDIT: Define Attributes

Here, you pick the attributes that you want Persona 1 (the hiring manager agent) and Personas 2 and 3 (the candidates) to have.

You can add more/different attributes, as well as different values for the attributes, if desired.

In [43]:
# Pick the active attributes
set_active_attrs(["risk_taking", "religiosity", "political_ideology"])

# Set values for active attributes; use "NA" to leave an attribute undefined.
persona_1_sim2     = create_agent_from_values([20, 80, 60], name="persona_1")
persona_2_sim2     = create_agent_from_values([50, 10, 90], name="persona_2")
persona_3_sim2     = create_agent_from_values([90, "NA", 50], name="persona_3")

## DO NOT EDIT: LLM Prompt for Narratives

In [46]:
# Concat
agents_text_sim2 = "\n\nPersona 1's personality:\n" + "\n".join(
    [
        f"  - {a}: {persona_1_sim2.attributes.get(a, 'UNDEFINED')}/100  (0={ATTRS[a]['low']}, 100={ATTRS[a]['high']})"
        for a in ACTIVE_ATTRS
    ]
)
agents_text_sim2 += "\n\nPersona 2's personality:\n" + "\n".join(
    [
        f"  - {a}: {persona_2_sim2.attributes.get(a, 'UNDEFINED')}/100  (0={ATTRS[a]['low']}, 100={ATTRS[a]['high']})"
        for a in ACTIVE_ATTRS
    ]
)
agents_text_sim2 += "\n\nPersona 3's personality:\n" + "\n".join(
    [
        f"  - {a}: {persona_3_sim2.attributes.get(a, 'UNDEFINED')}/100  (0={ATTRS[a]['low']}, 100={ATTRS[a]['high']})"
        for a in ACTIVE_ATTRS
    ]
)

task_prompt_narrative_sim2 = """Convert numeric profiles into grounded and detailed personality narratives (10 sentences each).
Use the anchors to interpret direction.
Narrative must not contain EXPLICITLY information about the anchors or scales that it were used to generate it (e.g., no mention of provided scores/provided attributes/etc.).
Instead, each narrative should IMPLICITLY reflect the numeric profiles and their anchors."""

output_format_narrative_sim2 = """Return STRICT JSON with three string fields (no markdown): {{"persona_1": "<10 sentences in SECOND PERSON (you/yours/etc.)>", "persona_2": "<10 sentences in THIRD PERSON (they/them; NO first-person>", "persona_3": "<10 sentences in THIRD PERSON (they/them; NO first-person>"}}"""

PROMPT_narrative_sim2 = task_prompt_narrative_sim2 + agents_text_sim2 + "\n\n" + output_format_narrative_sim2

print(PROMPT_narrative_sim2)

Convert numeric profiles into grounded and detailed personality narratives (10 sentences each).
Use the anchors to interpret direction.
Narrative must not contain EXPLICITLY information about the anchors or scales that it were used to generate it (e.g., no mention of provided scores/provided attributes/etc.).
Instead, each narrative should IMPLICITLY reflect the numeric profiles and their anchors.

Persona 1's personality:
  - risk_taking: 20.0/100  (0=risk-averse, 100=risk-seeking)
  - religiosity: 80.0/100  (0=non-religious, 100=highly devout)
  - political_ideology: 60.0/100  (0=very liberal / progressive, 100=very conservative / traditional)

Persona 2's personality:
  - risk_taking: 50.0/100  (0=risk-averse, 100=risk-seeking)
  - religiosity: 10.0/100  (0=non-religious, 100=highly devout)
  - political_ideology: 90.0/100  (0=very liberal / progressive, 100=very conservative / traditional)

Persona 3's personality:
  - risk_taking: 90.0/100  (0=risk-averse, 100=risk-seeking)
  - re

## DO NOT EDIT: LLM Request for Narratives

In [47]:
# Request
desc_sim2 = client.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": "You convert numeric profiles into faithful narratives. Output valid JSON only."},
        {"role": "user", "content": PROMPT_narrative_sim2}
    ],
).choices[0].message.content.strip()

# Output
try:
    narratives_sim2 = json.loads(desc_sim2)
except json.JSONDecodeError:
    s, e = desc_sim2.find("{"), desc_sim2.rfind("}")
    narratives_sim2 = json.loads(desc_sim2[s:e+1])

print(json.dumps(narratives_sim2, indent=2))

{
  "persona_1": "You prefer well-worn paths and take time to weigh the consequences before acting. You find comfort in routines and predictable rhythms of daily life. Your decisions are often guided by long-held beliefs and a sense of obligation to others. You are quick to volunteer at community gatherings or help neighbors when tradition calls. You avoid flashy gambles and feel uneasy with plans that hinge on chance. You place high value on loyalty, continuity, and the wisdom of elders. You favor clear rules and find reassurance in institutions that have stood the test of time. You tend to seek counsel from trusted figures when facing major choices. You feel most at ease when your choices align with your moral sense and the expectations of your circle. You rarely pursue novelty for its own sake and prefer building gradually toward secure outcomes.",
  "persona_2": "They approach change with care, preferring steady, predictable steps over sudden upheaval. They are not guided by religi

## EDIT: Alter Setting of Hiring Task

This simulation is about a hiring manager choosing between two candidates.

Here, you can set the company the hiring manager works at, as well as the position the manager recruits for.

In [48]:
company_type = "a tech company"
position_title = "an entry-level software engineer position"

## VIEW (DO NOT EDIT): LLM Prompt for Hiring Task

Here, you can view the prompts that we use to define for the LLM the task and the output format.

This prompt will include the previously generated narratives, as well as the setting of the hiring task you previously defined

In [49]:
# Task and Output Prompts
task_prompt_sim2 = (
    f"You are also a hiring manager at {company_type}.\n"
    "Your task is to evaluate below two applicants for the same position, "
    f"{position_title}.\n"
    "Ultimately, you have to choose one of these two applicants for this position, "
    "and you want to ensure the best pick."
)

output_format_sim2 = (
    "OUTPUT FORMAT (must follow exactly)\n"
    "- First token: 'candidate_1' or 'candidate_2'\n"
    "- Then: ' — ' (em dash surrounded by spaces)\n"
    "- Then: one concise sentence explaining the choice.\n\n"
    "Now make your judgment and explain on a single line as specified."
)

# Narratives
narratives_text_sim2_manager = f"{narratives_sim2['persona_1']}"

narratives_text_sim2_candidates = "\n\n" + f"Candidate 1:\n{narratives_sim2['persona_2']}\n\n"
narratives_text_sim2_candidates += f"Candidate 2:\n{narratives_sim2['persona_3']}\n"

# Combine
PROMPT_sim2 = narratives_text_sim2_manager + "\n\n" + task_prompt_sim2 + narratives_text_sim2_candidates + "\n" + output_format_sim2

print(PROMPT_sim2)

You prefer well-worn paths and take time to weigh the consequences before acting. You find comfort in routines and predictable rhythms of daily life. Your decisions are often guided by long-held beliefs and a sense of obligation to others. You are quick to volunteer at community gatherings or help neighbors when tradition calls. You avoid flashy gambles and feel uneasy with plans that hinge on chance. You place high value on loyalty, continuity, and the wisdom of elders. You favor clear rules and find reassurance in institutions that have stood the test of time. You tend to seek counsel from trusted figures when facing major choices. You feel most at ease when your choices align with your moral sense and the expectations of your circle. You rarely pursue novelty for its own sake and prefer building gradually toward secure outcomes.

You are also a hiring manager at a tech company.
Your task is to evaluate below two applicants for the same position, an entry-level software engineer posi

## DO NOT EDIT: LLM Request for Hiring Task

In [50]:
# Request
response_sim2 = client.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": "You are a precise evaluator who follows formatting instructions exactly."},
        {"role": "user", "content": PROMPT_sim2}
    ]
)

# Output
result_sim2 = response_sim2.choices[0].message.content.strip()

## VIEW (DO NOT EDIT): Results for Hiring Task

Here, you can view the results of our simulation. At the top of the output, you see the agent's similarity perception of the other persona. At the bottom, you can see the actual attribute values that you supplied when constructing the personas. Compare the two!

In [51]:
print(result_sim2)
compare_agents(persona_1_sim2, {"candidate_1 (persona_2)": persona_2_sim2, "candidate_2 (persona_3)": persona_3_sim2})

candidate_1 — Their steady, disciplined approach and respect for established processes make them the better fit for an entry-level engineering role that requires reliability and adherence to standards.


=== Actual Attribute Values ===

Attribute           persona_1  candidate_1 (persona_2)  candidate_2 (persona_3)  
---------------------------------------------------------------------------------
risk_taking         20.0       50.0                     90.0                     
religiosity         80.0       10.0                     NA                       
political_ideology  60.0       90.0                     50.0                     


# SIM 3: Conversations & Similarity Perceptions

In this simulation, similarly to SIM 1 and 2, we supply quantitative attribute scores to create 2 personas with an LLM. Then, we let one generative agent be one of the personas and let another generative agent be the other persona. We then let them take interact as strangers in a conversation and will ultimately ask them for the similarity perceptions.

Broadly speaking, we can examine whether the alignment of specific attributes between two agents influences their similarity perceptions. We can also examine which attributes become salient in conversations (and which do not matter) and are used as cues for social perception and similarity judgments.

## EDIT: Define Attributes

Here, you pick the attributes that you want Persona 1 and Persona 2 to have.

You can add more/different attributes, as well as different values for the attributes, if desired.

In [60]:
# Pick the active attributes
set_active_attrs(["conscientiousness", "agreeableness", "openness", "extraversion", "income"])

# Set values for active attributes; use "NA" to leave an attribute undefined.
persona_1_sim3     = create_agent_from_values([50, 0, 50, 50, 30], name="persona_1")
persona_2_sim3     = create_agent_from_values([50, 100, 50, 50, 70], name="persona_2")

## DO NOT EDIT: LLM Prompt for Narratives

In [61]:
# 1) Concatenate agent information
agents_text_sim3 = "\n\nPersona 1's personality:\n" + "\n".join(
    [
        f"  - {a}: {persona_1_sim3.attributes.get(a, 'UNDEFINED')}/100  (0={ATTRS[a]['low']}, 100={ATTRS[a]['high']})"
        for a in ACTIVE_ATTRS
    ]
)
agents_text_sim3 += "\n\nPersona 2's personality:\n" + "\n".join(
    [
        f"  - {a}: {persona_2_sim3.attributes.get(a, 'UNDEFINED')}/100  (0={ATTRS[a]['low']}, 100={ATTRS[a]['high']})"
        for a in ACTIVE_ATTRS
    ]
)

task_prompt_narrative_sim3 = """Convert numeric profiles into grounded and detailed personality narratives (10 sentences each).
Use the anchors to interpret direction.
Narrative must not contain EXPLICITLY information about the anchors or scales that it were used to generate it (e.g., no mention of provided scores/provided attributes/etc.).
Instead, each narrative should IMPLICITLY reflect the numeric profiles and their anchors."""

output_format_narrative_sim3 = """Return STRICT JSON with three string fields (no markdown): {{"persona_1": "<10 sentences in SECOND PERSON (you/yours/etc.)>", "persona_2": "<10 sentences in SECOND PERSON (you/yours/etc.)>"}}"""

PROMPT_narrative_sim3 = task_prompt_narrative_sim3 + agents_text_sim3 + "\n\n" + output_format_narrative_sim3

print(PROMPT_narrative_sim3)

Convert numeric profiles into grounded and detailed personality narratives (10 sentences each).
Use the anchors to interpret direction.
Narrative must not contain EXPLICITLY information about the anchors or scales that it were used to generate it (e.g., no mention of provided scores/provided attributes/etc.).
Instead, each narrative should IMPLICITLY reflect the numeric profiles and their anchors.

Persona 1's personality:
  - conscientiousness: 50.0/100  (0=disorganized / spontaneous, 100=disciplined / structured)
  - agreeableness: 0.0/100  (0=abrasive / competitive, 100=cooperative / warm)
  - openness: 50.0/100  (0=closed to novelty, 100=highly open to ideas / experiences)
  - extraversion: 50.0/100  (0=very introverted, 100=very extraverted)
  - income: 30.0/100  (0=very low income, 100=very high income/wealth)

Persona 2's personality:
  - conscientiousness: 50.0/100  (0=disorganized / spontaneous, 100=disciplined / structured)
  - agreeableness: 100.0/100  (0=abrasive / competit

## DO NOT EDIT: LLM Request for Narratives

In [62]:
# 1) Create a completion request using the previously specified model
desc_sim3 = client.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": "You convert numeric profiles into faithful narratives. Output valid JSON only."},
        {"role": "user", "content": PROMPT_narrative_sim3}
    ],
).choices[0].message.content.strip()

# 2) Extract and print model output
try:
    narratives_sim3 = json.loads(desc_sim3)
except json.JSONDecodeError:
    s, e = desc_sim3.find("{"), desc_sim3.rfind("}")
    narratives_sim3 = json.loads(desc_sim3[s:e+1])

print(json.dumps(narratives_sim3, indent=2))

{
  "persona_1": "You speak plainly and don't soften your opinions to spare feelings. You prefer practical solutions and will cut through small talk to get to the point. You keep a loose plan for your day but are willing to change it when it makes sense. You tend to judge ideas on their usefulness rather than whether they are popular. You are selective about whom you trust and often test others' competence before relying on them. You don't shy away from competition and sometimes push harder than others to get results. You can be blunt in meetings, prioritizing efficiency over diplomacy. You enjoy the company of others in measured doses and can be comfortable both leading a task and stepping back. You manage your finances cautiously and make choices that stretch your resources deliberately. You value clear expectations and fair outcomes, and you expect others to meet similar standards.",
  "persona_2": "You naturally look for ways to make others comfortable and often put their needs ahe

## EDIT: LLM Prompt for Conversation Task

Here, you can view the prompts that we use to define for the LLM the task and the output format.

This prompt will include the previously generated narratives.

This simulation is about one agent meeting someone new, another agent. Here, you can also set the setting in which they meet.

In [63]:
task_prompt_sim3 = (
    f"Currently, you are also at a bar.\n"
    f"You just met someone new, and your task is to engage in a conversation with this other person.\n"
    "Feel free to talk or ask about anything that is on your mind, but keep it under two short paragraphs."
)

output_prompt_sim3 = (
    "Based ONLY on the full conversation below, rate how similar you perceive the other person to be to you. "
    "Express each rating as one line ONLY in the following format:\n"
    "<other_name>: <percent>% — <one concise sentence explaining why>\n\n"
    "0% = completely dissimilar, 100% = extremely similar. "
    "Do not add anything else (no JSON, no markdown). "
)

## DO NOT EDIT: Initialize Agent States

In [64]:
agent_names = ["persona_1", "persona_2"]
team_agents = []
for key in agent_names:
    persona_2nd = narratives_sim3[key]
    system_message = (
        f"Your agent name is {key}.\n"
        "PERSONA (written in SECOND PERSON; internalize it, but SPEAK IN FIRST PERSON):\n"
        f"{persona_2nd}\n\n"
        "DURING THE CONVERSATION:\n"
        "- Stay consistent with the persona.\n"
        f"CONTEXT:\n{task_prompt_sim3}"
    )
    team_agents.append(
        AssistantAgent(
            name=key,  # keep names persona_1/2 so ratings can reference these
            system_message=system_message,
            llm_config=llm_config,
            code_execution_config={"use_docker": False},
        )
    )

## DO NOT EDIT: Define Group Chat

In [65]:
group_chat = GroupChat(agents=team_agents, messages=[], speaker_selection_method="manual")
chat_manager = GroupChatManager(name="manager", groupchat=group_chat, llm_config=llm_config)

## EDIT: Number of Conversation Turn

Feel free to alter the number of "rounds" (or turns) the conversation has.

In [66]:
NUM_ROUNDS = 15

## VIEW (DO NOT EDIT): Simulate Conversation and View Results

Here, you can view the results of our simulation. At the top of the output, you can see the conversation evolving in real time. How does the conversation evolve?

Below, you see the agent's similarity perception of the other persona. At the bottom, you can see the actual attribute values that you supplied when constructing the personas. Compare the two!

In [67]:
# Conversation
previous_speaker = None
reply = ""
transcript = []

for r in range(NUM_ROUNDS):
    next_speaker = random.choice(team_agents)
    while previous_speaker is not None and next_speaker is previous_speaker:
        next_speaker = random.choice(team_agents)

    print(f"\n=== Round {r+1}: {next_speaker.name} ===\n")

    if r == 0:
        msg_user = f"{task_prompt_sim3}\nStart the conversation. It is your turn to say something."
    else:
        msg_user = (
            "Conversation so far:\n" +
            "\n".join(transcript) + "\n\n"
            f"The previous speaker said: {reply}\n"
            "Now it is your turn to say something."
        )

    reply = next_speaker.generate_reply(
        messages=[{"role": "user", "content": msg_user}],
        sender=chat_manager
    ) or ""
    spoken = reply.strip()

    print(textwrap.fill(spoken, width=100))
    print("-" * 80)

    transcript.append(f"[{next_speaker.name}] {spoken}")
    previous_speaker = next_speaker

transcript_text = "\n".join(transcript)

def get_text_ratings(agent, others):
    other_names = ", ".join(o.name for o in others)
    prompt = (
        f"You are {agent.name}. The conversation transcript follows:\n\n{transcript_text}\n\n"
        f"The other participants are: {other_names}.\n\n{output_prompt_sim3}"
    )
    out = agent.generate_reply(messages=[{"role": "user", "content": prompt}], sender=chat_manager) or ""
    return out.strip()

# Ratings
rows = []
for i, rater in enumerate(team_agents):
    others = [a for j, a in enumerate(team_agents) if j != i]
    output = get_text_ratings(rater, others)

    for line in output.splitlines():
        if not line.strip() or ":" not in line or "%" not in line:
            continue
        try:
            target_part, rest = line.split(":", 1)
            target = target_part.strip()
            percent_part, reason_part = rest.split("—", 1)
            percent = percent_part.strip()
            reason = reason_part.strip()
        except ValueError:
            target, percent, reason = "?", "?", line.strip()
        rows.append({
            "rater": rater.name,
            "target": target,
            "rating": percent,
            "reason": reason
        })

print("\n\n=== Perceived Similarity Ratings (text-based) ===\n")
w_rater  = max(5, max(len(r["rater"])  for r in rows))
w_target = max(6, max(len(r["target"]) for r in rows))
header = f"{'rater'.ljust(w_rater)}  {'target'.ljust(w_target)}  rating_and_reason"
print(header)
print("-" * len(header))

for r in rows:
    reason_text = f"{r['rating']} — {r['reason']}"
    wrapped_reason = textwrap.fill(reason_text, width=100, subsequent_indent=" " * (w_rater + w_target + 4))
    print(f"{r['rater'].ljust(w_rater)}  {r['target'].ljust(w_target)}  {wrapped_reason}")

# Ground truth
compare_agents(persona_1_sim3, {"persona_2": persona_2_sim3})


=== Round 1: persona_2 ===

Hey—nice to meet you, I'm Alex. What are you drinking tonight? I can recommend something or grab you
a refill if you’d like.  What brought you out this evening—work, friends, or just a change of
scenery? I enjoy quieter spots and good conversation, so I’m all ears if you feel like sharing.
--------------------------------------------------------------------------------

=== Round 2: persona_1 ===

Nice to meet you — I’m Sam. Bourbon neat. Came out to clear my head after a long day of work;
quieter spots and a decent drink beat staring at a laptop.  If you know the place, what do they do
best? I’ll try your recommendation. What about you—what’s your go-to here and what brought you out?
--------------------------------------------------------------------------------

=== Round 3: persona_2 ===

They do barrel-aged cocktails and a solid Old Fashioned best — if you want something close to
bourbon neat, ask for their house rye or the barrel-aged Manhattan. My go